# Lezione 11

In [ ]:
from liblet import Grammar, Derivation, cyk2table
from L09 import cyk, to_cnf

## Simboli e input: stringhe e sequenze

Un "chiarimento" dovuto su *simboli* e *input*.

In questi *handout* i **simboli** (siano essi terminali che non) sono rappresentati da stringhe, o caratteri, che in entrambe i casi corrispondono al tipo [Text Sequence Type](https://docs.python.org/3.7/library/stdtypes.html#text-sequence-type-str) (brevemente `str`, osservate che in Python *non esiste il tipo carattere*). 

Questo è evidente se, ad esempio, si ispeziona una `Grammar`

In [ ]:
G = Grammar.from_string("""
Name -> First Last
First -> mario | franco
Last -> bruni | rossi 
""")

In [ ]:
# Sono stringhe i non terminali

set(G.N)

{'First', 'Last', 'Name'}

In [ ]:
# ma pure i terminali!

set(G.T)

{'bruni', 'franco', 'mario', 'rossi'}

In base alla definizione di **linguaggio** una **parola** (o **forma sentenziale**) è una sequenza di simboli, ragion per cui la rappresentazione naturale di una parola è quella di una [lista](https://docs.python.org/3.7/tutorial/datastructures.html#more-on-lists), o [tupla](https://docs.python.org/3.7/tutorial/datastructures.html#tuples-and-sequences).

Di nuovo, questo è evidente se, ad esempio, si ispeziona la forma sentenziale di una `Derivation`

In [ ]:
Derivation(G).leftmost((0, 2, 4)).sentential_form()

('franco', 'rossi')

In linea di principio, quindi, l'*input* (termine informale) dei nostri algoritmi che corrisponde al termine formale *parola* deve essere una sequenza di simboli, quindi una lista, o tupla.

Non ha senso pensare all'*input* come ad una stringa, perché questo impedisce di riconoscervi (in modo non "ambiguo") i simboli.

Badate bene che è molto diverso dire che `('franco', 'franco')` è una parola, ma non appartiene al linguaggio generato da `G`, dal dire che `'francofranco'` non solo non appartiene a tale linguaggio, ma non è nemneno una parola (su `G.T`) dal momento che, pur essendo una stringa di lettere, non è una sequenza di simboli terminali di `G.T`.

In [ ]:
G_cnf = to_cnf(G)

In [ ]:
# 'francofranco' conduce ad una tabella di parsing 
# col numero errato di celle (dovrebbe essere 2 x 2)

cyk2table(cyk(G_cnf, 'francofranco'))

In [ ]:
# ()'franco', 'franco') conduce ad una tabella di parsing 
# 2 x 2 che nella cella più alta non ha il simbolo di partenza
# dato che la parola non appartiene al linguaggio generato da G

cyk2table(cyk(G_cnf, 'francofranco'))
cyk2table(cyk(G_cnf, ('franco', 'franco')))

### Se i simboli sono tutti lunghi uno?

Può accadere (ed è accaduto senza che lo rendessi esplicito, cosa che può avervi indotti in confusione), che se i simboli terminali sono tutti rappresentati da stringhe di lunghezza uno, l'*input* possa essere "impropriamente" rappresentato invece che con una lista di stringhe di lunghezza uno, come una stringa. 

Questa "confusione" tra i tipi è resa possibile dal fatto che in Python l'espressione `INPUT[i]` è legittima sia nel caso in cui `INPUT` si riferisca ad una stringa, che ad una lista, o squenza.

In [ ]:
# rappresentazione propria

INPUT = ('p', 'i', 'p', 'p', 'o')

INPUT[1]

'i'

In [ ]:
# rappresentazione impropria

INPUT = 'pippo'

INPUT[1]

'i'

## Tokenizzazione e parsing

La confusione si può fare ancora più acuta quando inizieremo a parlare di tokenizzazione, oltre che di parsing.

In tal caso ci sono due grammatiche (e due linguaggi), in gioco.

La prima grammatica $G_T = (N_T, T_T, P_T, S_T)$ è quella del **tokenizzatore**, è in generale una grammatica regolare i cui terminali $T_T$ sono i caratteri dell'alfabeto di macchina (ad esempio i caratteri *Unicode*). 

Tale grammatica può essere pensata come l'unione di $k>0$ grammatiche regolari $G^k_T = (N^k_T, T_T, P^k_T, S^k_T)$ (in cui gli $N^k_T$ e $P^k_T$ sono disgiunti e gli $S^k_T$ sono distinti), ciascuna delle quali riconosce un certo tipo di *token*. $G_T$ è usualmente definita da $N_T =  \{S_T\} \cup \bigcup N^k_T$ e $P_T =  \{S_T \to ( S^1_T | S^2_T |  \ldots | S^k_T )^* \} \cup \bigcup P^k_T$ (dove la produzione per il simbolo distinto, qui descritta con la notazione impropria delle espressioni regolari, indica che le parole $G_T$ sono sequenze di parole delle varie $G^k_T$, ossia di token.

La seconda grammatica $G_P = (N_P, T_P, P_P, S_P)$ è quella del **parser**, è in generale una grammatica libera da contesto i cui terminali $T_P$ sono i simboli distinti delle grammatiche $G^k_T$, ossia $T_P = \{S^1_T, S^2_T, \ldots, S^k_T\}$. 

In questo contesto, parlare di *input* può condurre ad ancor maggior cnfusione: tale termine può riferirsi sia ad una parola in $T_T^*$ che il tokenizzatore trasformerà in una parola in $T_P^*$, che a quest'ultima parola che sarà l'*input* del parser (che lo trasformerà in un albero di parsing su $G_P$). Dopo di che avverrà la traduzione (interpretazione, o compilazione che sia)…

### Un esempio

Facciamo un esempio concreto, per chiarirci le idee. Supponiamo di voler scrivere una calcolatrice in grado di compiere operazioni aritmetiche su interi e veriabili, come ad esempio `52* pi`.

Dapprima avremo bisogno di dividere l'espressione in token, per farlo avremo bisogno di tre grammatiche regolari, una per gli interi: `Number -> [0-9]+`, una per le variabili `Var -> [a-z]+` e una per gli operatori `Op -> +|-|*|/`. Le metteremo assieme nella grammatica del tokenizzatore che avrà come prima produzione `S -> (' ' | Number | Var | Op)*` (osservate che c'è anche un "trucco" per buttare via gli spazi bianchi).
 
Ora avremo bisogno di una grammatica libera da contesto per le espressioni: `Expr -> Expr Op Expr | Number | Var` (il cui unico non terminale è il simbolo di partenza `Expr`).

Siamo pronti: dato l'input (`5`, `2`, `*`, ` `, `p`, `i`) sull'alfabeto Unicode, il tokenizzatore lo trasforma nell'input (`Number`, `Op`, `Var`) sull'alfabeto dei terminali del parser che quest'ultimo determinerà essere prodotto dalla derivazione leftmost `Expr -> Expr Op Expr | Number Op Expr -> Number Op Var`.

Ora vi domanderete, è il valore dell'espressione? Questa è una ulteriore "complicazione". A ciascun *token* è associato un valore, che dipende dalla porzione di stringa che deriva (sull'alfabeto $T_T$). Ad esempio, il valore del primo token è 52, il valore del secondo è "moltiplicazione" mentre il valore del terzo ed ultimo è `pi`.

# <span style="color: red">Esercizi per casa</span>

* Implementare tre funzioni che, dato un automa per $L_1$ e $L_2$, costruiscano rispettivamente l'automa (non deterministico) per $L_1L_2$, $L_1\cup  L_2$ e $L_1^*$.

* Implementare una funzione per la costruzione dell'*automa completo* secondo l'algoritmo presentato nella Sezione 5.5; usatelo per scrivere una funzione che costruisca l'*automa complemento* per $L^C_1 = T^* - L_1$.

* Implementate una funzione per la costruzione dell'*automa intersezione* per $L_1\cap L_2$ secondo l'algoritmo presentato nella Sezione 5.5; provate a confrontarne il comportamento di questo automa con quello per $(L^C_1 \cup L^C_2)^C$ costruito con le funzioni del punto precedente.

* Implementate una funzione per la *minimizzazione* di un DFA, o secondo l'algoritmo presentato nella Sezione 5.7, oppure secondo l'algoritmo di [Brzozowski](https://en.wikipedia.org/wiki/DFA_minimization#Brzozowski's_algorithm).